# Testing the DEB model
## Santruckova et al. (unpublished data)
1. Importing libraries

In [52]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

2. Defining DEB model

In [122]:
def DEBmodel (y, t, pars):
    #define initial pools
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3]
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    ce=pars[5];
    MX1=ce/4;
    #Define fluxes
    ##scaling function for substrate
    f=S/(Km+S)
    uptake=(v*ce/yA)*X1**(2/3)*f
    growth = (v*e*X1**(-1/3)-m*g)/(e+g)
    #Define derivatives
    dSdt = -uptake
    dedt = v*X1**(-1/3)*(f - e)
    dX1dt = X1*growth 
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v*X1**(-1/3)-growth)) - growth*X1*MX1
    return dSdt, dedt, dX1dt, dCO2dt;

3. Defining outputs from DEB model

In [123]:
def calcDEB (model, pars, t, y0):
    #model parameters
    pars_model=pars[0:6]
    #conversion factors
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate B, WallsProto,
    B=((conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2])
    WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])
    #Create data with predictions
    yhat = np.concatenate((y[:, 3].reshape(len(d.Time),1),#CO2
                           B.reshape(len(d.Time),1),
                           WallsProto.reshape(len(d.Time),1)), axis=1)
    
    return yhat

4. Defining objective function

In [124]:
def obj_funDEB (x):
    #define parameters
    pars = x
    #initial conditions
    S_i = d.Sinit[0]
    e_i =pars[6]/4/d.WP[0]
    X1_i = d.B[0]/(pars[5]*(0.25 + e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.Rc]).reshape(len(d.Time),1),
                           np.array([d.B]).reshape(len(d.Time),1),
                           np.array([d.WP]).reshape(len(d.Time),1)), axis=1)
    #weights
    weights=np.concatenate((np.nanmean(d.Rc).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.B).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((d.WP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    out=np.nansum(((yhat_full-obs)/weights)**2)
    return out

5. Calculate the goodness of fit

In [125]:
def goodnessDEB (x):
    #define parameters
    pars = x
    #initial conditions
    S_i = d.Sinit[0]
    e_i =pars[6]/4/d.WP[0]
    X1_i = d.B[0]/(pars[5]*(0.25 + e_i))
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.Rc]).reshape(len(d.Time),1),
                           np.array([d.B]).reshape(len(d.Time),1),
                           np.array([d.WP]).reshape(len(d.Time),1)), axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    out = np.array([R2, ll, AIC])
    return out

6. Reading data

In [62]:
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/VidenSucrose.csv', sep=',')
print(d)

    Time  Sinit         Rc         B        WP
0    0.1  51.29   0.110000  2.130000  0.054500
1   24.0  51.29  15.831667  3.590000  0.359000
2   48.0  51.29  22.078333  3.723333  0.718833
3   72.0  51.29  24.503333  3.223333  0.624000
4   96.0  51.29  26.570000  2.593333  1.051833
5  120.0  51.29  27.845000  1.733333  0.496000


7. Estimating parameters

In [126]:
VidenSucroseP=dual_annealing(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 30), (0, 1)])

<ipython-input-123-de0bdb1e2a70>:11: RuntimeWarning: invalid value encountered in true_divide
  WallsProto = (conversions[1]*conversions[0]/4)/(conversions[0]*y[:, 1])


In [127]:
print(VidenSucroseP)
print(goodnessDEB(VidenSucroseP.x))

     fun: 1.4150606906953584
 message: ['Maximum number of iteration reached']
    nfev: 31785
    nhev: 0
     nit: 1000
    njev: 2223
  status: 0
 success: True
       x: array([9.87142673e-01, 3.84059558e+02, 2.46062017e-01, 9.01407932e-02,
       3.37332001e-01, 2.99708325e+01, 1.75215197e-01])
[ 0.90459578 -0.85863794 15.71727587]


In [128]:
VidenSucroseDE=differential_evolution(obj_funDEB, [(0.05, 1), (10, 1000),(0.001, 5), (1e-12, 0.1), (0.1, 3), (0.1, 30), (0, 1)])

In [129]:
print(VidenSucroseDE)
print(goodnessDEB(VidenSucroseDE.x))

     fun: 1.3307232014617958
     jac: array([ 7.40518754e-05,  4.90052054e-05,  9.67292913e-04, -5.79103432e-03,
       -1.77606818e-03,  3.69038103e-05,  3.33055805e-03])
 message: 'Optimization terminated successfully.'
    nfev: 14462
     nit: 133
 success: True
       x: array([5.22690339e-01, 9.95743451e+02, 2.94003210e-01, 6.89088743e-02,
       2.00229849e-01, 2.72128319e+01, 6.97450646e-02])
[ 0.9185374  -0.73316336 15.46632671]


In [130]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/VidenSucroseParsSV.csv', VidenSucroseDE.x.reshape(1,7), delimiter=",")